In [1]:
from __future__ import print_function, division


from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.datasets import cifar10

import matplotlib.pyplot as plt

import sys

import numpy as np

In [10]:
class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, 1, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()
        

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [13]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 3072)              0         
                                                                 
 dense_42 (Dense)            (None, 512)               1573376   
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_43 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_44 (Dense)            (None, 1)                 257       
                                                                 
Total params: 1,704,961
Trainable params: 1,704,961
N

91 [D loss: 0.496163, acc.: 70.31%] [G loss: 1.815371]
92 [D loss: 0.724162, acc.: 56.25%] [G loss: 1.737530]
93 [D loss: 0.601663, acc.: 65.62%] [G loss: 1.726344]
94 [D loss: 0.549284, acc.: 73.44%] [G loss: 1.774852]
95 [D loss: 0.662646, acc.: 60.94%] [G loss: 1.664770]
96 [D loss: 0.570659, acc.: 75.00%] [G loss: 1.711994]
97 [D loss: 0.706098, acc.: 65.62%] [G loss: 1.443123]
98 [D loss: 0.621147, acc.: 62.50%] [G loss: 1.379848]
99 [D loss: 0.464985, acc.: 79.69%] [G loss: 1.640733]
100 [D loss: 0.515674, acc.: 71.88%] [G loss: 1.818672]
101 [D loss: 0.678379, acc.: 62.50%] [G loss: 1.899527]
102 [D loss: 0.709275, acc.: 65.62%] [G loss: 1.662764]
103 [D loss: 0.729949, acc.: 62.50%] [G loss: 1.586493]
104 [D loss: 0.652915, acc.: 62.50%] [G loss: 1.456138]
105 [D loss: 0.628728, acc.: 68.75%] [G loss: 1.711051]
106 [D loss: 0.682190, acc.: 71.88%] [G loss: 1.576582]
107 [D loss: 0.521646, acc.: 78.12%] [G loss: 1.765371]
108 [D loss: 0.535758, acc.: 76.56%] [G loss: 1.708462]
1

241 [D loss: 0.779262, acc.: 50.00%] [G loss: 1.406475]
242 [D loss: 0.679374, acc.: 67.19%] [G loss: 1.310022]
243 [D loss: 0.689776, acc.: 62.50%] [G loss: 1.339084]
244 [D loss: 0.475338, acc.: 84.38%] [G loss: 1.416477]
245 [D loss: 0.727350, acc.: 64.06%] [G loss: 1.462684]
246 [D loss: 0.686798, acc.: 59.38%] [G loss: 1.622811]
247 [D loss: 0.622755, acc.: 71.88%] [G loss: 1.493629]
248 [D loss: 0.703119, acc.: 50.00%] [G loss: 1.433586]
249 [D loss: 0.713744, acc.: 60.94%] [G loss: 1.528426]
250 [D loss: 0.688831, acc.: 62.50%] [G loss: 1.224792]
251 [D loss: 0.596643, acc.: 65.62%] [G loss: 1.313517]
252 [D loss: 0.547125, acc.: 71.88%] [G loss: 1.335544]
253 [D loss: 0.720044, acc.: 59.38%] [G loss: 1.356223]
254 [D loss: 0.600589, acc.: 67.19%] [G loss: 1.575101]
255 [D loss: 0.761840, acc.: 46.88%] [G loss: 1.281103]
256 [D loss: 0.776462, acc.: 56.25%] [G loss: 1.333826]
257 [D loss: 0.636449, acc.: 59.38%] [G loss: 1.390980]
258 [D loss: 0.662694, acc.: 60.94%] [G loss: 1.

391 [D loss: 0.654284, acc.: 51.56%] [G loss: 1.248954]
392 [D loss: 0.600865, acc.: 65.62%] [G loss: 1.563753]
393 [D loss: 0.729243, acc.: 59.38%] [G loss: 1.213249]
394 [D loss: 0.707542, acc.: 53.12%] [G loss: 1.273117]
395 [D loss: 0.621054, acc.: 65.62%] [G loss: 1.331350]
396 [D loss: 0.659285, acc.: 62.50%] [G loss: 1.304077]
397 [D loss: 0.610685, acc.: 65.62%] [G loss: 1.265657]
398 [D loss: 0.644601, acc.: 67.19%] [G loss: 1.269023]
399 [D loss: 0.529465, acc.: 68.75%] [G loss: 1.492241]
400 [D loss: 0.669573, acc.: 60.94%] [G loss: 1.496486]
401 [D loss: 0.738831, acc.: 56.25%] [G loss: 1.341681]
402 [D loss: 0.623371, acc.: 65.62%] [G loss: 1.408472]
403 [D loss: 0.693447, acc.: 56.25%] [G loss: 1.327565]
404 [D loss: 0.561779, acc.: 73.44%] [G loss: 1.459808]
405 [D loss: 0.634628, acc.: 62.50%] [G loss: 1.396130]
406 [D loss: 0.734173, acc.: 53.12%] [G loss: 1.316253]
407 [D loss: 0.695770, acc.: 53.12%] [G loss: 1.310056]
408 [D loss: 0.726462, acc.: 56.25%] [G loss: 1.

541 [D loss: 0.650944, acc.: 64.06%] [G loss: 1.481832]
542 [D loss: 0.756008, acc.: 53.12%] [G loss: 1.301392]
543 [D loss: 0.748330, acc.: 51.56%] [G loss: 1.189073]
544 [D loss: 0.570185, acc.: 65.62%] [G loss: 1.350635]
545 [D loss: 0.636563, acc.: 60.94%] [G loss: 1.587249]
546 [D loss: 0.695303, acc.: 50.00%] [G loss: 1.431641]
547 [D loss: 0.652639, acc.: 54.69%] [G loss: 1.307226]
548 [D loss: 0.745606, acc.: 43.75%] [G loss: 1.244768]
549 [D loss: 0.629818, acc.: 64.06%] [G loss: 1.365120]
550 [D loss: 0.630171, acc.: 67.19%] [G loss: 1.440653]
551 [D loss: 0.597310, acc.: 67.19%] [G loss: 1.472478]
552 [D loss: 0.807683, acc.: 53.12%] [G loss: 1.245416]
553 [D loss: 0.608142, acc.: 57.81%] [G loss: 1.281397]
554 [D loss: 0.687275, acc.: 57.81%] [G loss: 1.335938]
555 [D loss: 0.691373, acc.: 62.50%] [G loss: 1.307903]
556 [D loss: 0.631989, acc.: 70.31%] [G loss: 1.172674]
557 [D loss: 0.641168, acc.: 64.06%] [G loss: 1.223700]
558 [D loss: 0.539278, acc.: 75.00%] [G loss: 1.

...

29411 [D loss: 0.405357, acc.: 81.25%] [G loss: 3.646900]
29412 [D loss: 0.353433, acc.: 85.94%] [G loss: 3.631619]
29413 [D loss: 0.253400, acc.: 90.62%] [G loss: 3.097367]
29414 [D loss: 0.304309, acc.: 82.81%] [G loss: 3.645236]
29415 [D loss: 0.272764, acc.: 90.62%] [G loss: 3.068142]
29416 [D loss: 0.485103, acc.: 78.12%] [G loss: 3.225111]
29417 [D loss: 0.268063, acc.: 89.06%] [G loss: 4.483655]
29418 [D loss: 0.223788, acc.: 90.62%] [G loss: 2.780269]
29419 [D loss: 0.362368, acc.: 84.38%] [G loss: 2.780440]
29420 [D loss: 0.248415, acc.: 93.75%] [G loss: 3.092241]
29421 [D loss: 0.394559, acc.: 78.12%] [G loss: 2.009687]
29422 [D loss: 0.347935, acc.: 81.25%] [G loss: 2.997375]
29423 [D loss: 0.305434, acc.: 89.06%] [G loss: 2.967171]
29424 [D loss: 0.250301, acc.: 85.94%] [G loss: 3.084785]
29425 [D loss: 0.436686, acc.: 76.56%] [G loss: 2.328752]
29426 [D loss: 0.338957, acc.: 84.38%] [G loss: 3.794982]
29427 [D loss: 0.298263, acc.: 87.50%] [G loss: 3.371829]
29428 [D loss:

29556 [D loss: 0.224390, acc.: 89.06%] [G loss: 3.054278]
29557 [D loss: 0.351902, acc.: 81.25%] [G loss: 2.629890]
29558 [D loss: 0.137368, acc.: 98.44%] [G loss: 4.027161]
29559 [D loss: 0.405386, acc.: 78.12%] [G loss: 2.989439]
29560 [D loss: 0.236232, acc.: 87.50%] [G loss: 3.161046]
29561 [D loss: 0.397212, acc.: 81.25%] [G loss: 2.856221]
29562 [D loss: 0.357941, acc.: 87.50%] [G loss: 3.878386]
29563 [D loss: 0.247552, acc.: 92.19%] [G loss: 3.434007]
29564 [D loss: 0.336396, acc.: 85.94%] [G loss: 3.454813]
29565 [D loss: 0.422389, acc.: 76.56%] [G loss: 2.278318]
29566 [D loss: 0.370912, acc.: 87.50%] [G loss: 2.872953]
29567 [D loss: 0.344997, acc.: 82.81%] [G loss: 3.095165]
29568 [D loss: 0.205934, acc.: 92.19%] [G loss: 3.187106]
29569 [D loss: 0.324392, acc.: 85.94%] [G loss: 4.444423]
29570 [D loss: 0.374028, acc.: 81.25%] [G loss: 2.826535]
29571 [D loss: 0.365753, acc.: 79.69%] [G loss: 4.073802]
29572 [D loss: 0.207896, acc.: 92.19%] [G loss: 2.541399]
29573 [D loss:

29701 [D loss: 0.234837, acc.: 90.62%] [G loss: 3.190974]
29702 [D loss: 0.315636, acc.: 85.94%] [G loss: 3.143692]
29703 [D loss: 0.256152, acc.: 90.62%] [G loss: 3.907729]
29704 [D loss: 0.289217, acc.: 90.62%] [G loss: 3.379703]
29705 [D loss: 0.288408, acc.: 85.94%] [G loss: 3.528813]
29706 [D loss: 0.342542, acc.: 87.50%] [G loss: 2.361178]
29707 [D loss: 0.367389, acc.: 84.38%] [G loss: 2.218573]
29708 [D loss: 0.324532, acc.: 85.94%] [G loss: 2.836457]
29709 [D loss: 0.318571, acc.: 85.94%] [G loss: 2.234700]
29710 [D loss: 0.507787, acc.: 75.00%] [G loss: 2.584931]
29711 [D loss: 0.305073, acc.: 84.38%] [G loss: 2.804019]
29712 [D loss: 0.206702, acc.: 95.31%] [G loss: 2.321321]
29713 [D loss: 0.276510, acc.: 84.38%] [G loss: 2.778769]
29714 [D loss: 0.379968, acc.: 79.69%] [G loss: 4.727915]
29715 [D loss: 0.248435, acc.: 90.62%] [G loss: 4.292515]
29716 [D loss: 0.332738, acc.: 84.38%] [G loss: 9.016677]
29717 [D loss: 0.200897, acc.: 90.62%] [G loss: 6.238790]
29718 [D loss:

29846 [D loss: 0.366993, acc.: 81.25%] [G loss: 2.660574]
29847 [D loss: 0.356741, acc.: 81.25%] [G loss: 2.995789]
29848 [D loss: 0.443642, acc.: 79.69%] [G loss: 3.837547]
29849 [D loss: 0.424989, acc.: 79.69%] [G loss: 3.003589]
29850 [D loss: 0.249695, acc.: 90.62%] [G loss: 2.087142]
29851 [D loss: 0.253779, acc.: 92.19%] [G loss: 3.073628]
29852 [D loss: 0.374315, acc.: 84.38%] [G loss: 2.864882]
29853 [D loss: 0.275774, acc.: 89.06%] [G loss: 2.597233]
29854 [D loss: 0.403677, acc.: 81.25%] [G loss: 3.308597]
29855 [D loss: 0.280676, acc.: 84.38%] [G loss: 2.948141]
29856 [D loss: 0.306270, acc.: 85.94%] [G loss: 2.341247]
29857 [D loss: 0.343507, acc.: 92.19%] [G loss: 2.877480]
29858 [D loss: 0.357174, acc.: 82.81%] [G loss: 2.673837]
29859 [D loss: 0.291262, acc.: 87.50%] [G loss: 3.314573]
29860 [D loss: 0.329082, acc.: 82.81%] [G loss: 3.387243]
29861 [D loss: 0.336657, acc.: 85.94%] [G loss: 3.602098]
29862 [D loss: 0.351902, acc.: 82.81%] [G loss: 2.831251]
29863 [D loss:

29991 [D loss: 0.291888, acc.: 89.06%] [G loss: 3.037294]
29992 [D loss: 0.383062, acc.: 87.50%] [G loss: 3.099048]
29993 [D loss: 0.306598, acc.: 90.62%] [G loss: 2.799755]
29994 [D loss: 0.403077, acc.: 82.81%] [G loss: 3.150231]
29995 [D loss: 0.381299, acc.: 79.69%] [G loss: 4.215181]
29996 [D loss: 0.398912, acc.: 79.69%] [G loss: 2.826558]
29997 [D loss: 0.231941, acc.: 87.50%] [G loss: 4.619992]
29998 [D loss: 0.362167, acc.: 84.38%] [G loss: 2.504330]
29999 [D loss: 0.317823, acc.: 90.62%] [G loss: 2.431522]
